# Set UP

In [1]:
import os
from pprint import pprint
from IPython.display import Markdown, display
from langfuse.openai import AsyncOpenAI
from dotenv import load_dotenv
load_dotenv()
client = AsyncOpenAI()

# from langfuse.openai import openai
# from openai import OpenAI

In [2]:
def _msg(role, content): #returns a dict
    return {'role':role, 'content':content}

def user(content):
    return _msg('user', content)

def system(content):
    return _msg('system', content)

def assistant(content):
    return _msg('assistant', content)

# Caching

In [3]:
from diskcache import Cache
# cache = Cache() --> this is a temporary cache where it's not saved when the program is restart
cache = Cache(directory = '.cache_coures_dana') # This is not a temporary where tha cache is saved in a directory
                                                # and can be accessed any time


In [4]:
# make the caching get and set Asyc:
# wrapper thing
# when making async function: 'async' key word, when calling it, 'await' key word
import asyncio
async def set_async_cache(key, val, **kwargs): # needs key and value, and it can take other args by their key words(kwargs)
    # return sth that is awiat
    return await asyncio.to_thread(cache.set, key, val, **kwargs)

async def get_async_cache(key, **kwargs): # needs key, kwargs, and a default can be passed with kwargs
    return await asyncio.to_thread(cache.get, key, **kwargs)

# to call async func must be await

### Make the cache key with hashing

In [5]:
from hashlib import md5
import json
# print(md5(b'dana the one').hexdigest())
# md5 works with only str

def cache_key(key, **kwargs):
    kwargs_str = json.dumps(kwargs, sort_keys = True)
    kwargs_hashed = md5(kwargs_str.encode('utf-8')).hexdigest()
    hashed_key = f'{key}__{kwargs_hashed}'
    return hashed_key

def _chat_completion_cache_key(*, model, messages, **kwargs): # the first * to make the function only accepts key words args
    return cache_key('openai_chat_completion', model = model, messages = messages, **kwargs)

In [6]:
from openai.types.chat import ChatCompletion
from functools import update_wrapper
CACHE_MISS_SENTINEL = object() # we need to create sth that can be created ONLY once and can't be created accidentally anytime, like a memory address => an object

async def cached_chat_completion(*, model, messages, **kwargs) -> ChatCompletion : # the 'async' because we want to use async functions
    # MAKE KEY
    cache_key = _chat_completion_cache_key(model = model, messages = messages, **kwargs)
    cached_value = await get_async_cache(cache_key, default = CACHE_MISS_SENTINEL) # we will use this as default when the key can't be found in the cache
    
    # CACHE MISS
    if cached_value is CACHE_MISS_SENTINEL:
        completion = await client.chat.completions.create(model = model, messages = messages, **kwargs) # a) make openai call
        await set_async_cache(cache_key, completion.model_dump_json()) # b) set the output in the cache
        return completion # this is a chat completion type

    # CACHE HIT
    else:
        return ChatCompletion.model_validate(json.loads(cached_value)) # Cached Value is the vlue we need

cached_chat_completion = update_wrapper(cached_chat_completion, client.chat.completions.create) # for auto completion

# The Project

### Beautiful Soup

In [7]:
import requests
from bs4 import BeautifulSoup

In [8]:
class Website:
    url: str
    title: str
    content: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "Title not found"

        for irrelevant in soup.body(['script','style','img', 'input']):
            irrelevant.decompose() # to remove these tags from the body

        self.content = soup.body.get_text(separator = '\n', strip = True)

In [9]:
lookmovie = Website('https://www.lookmovie2.to/movies/page/6')
print(lookmovie.title)

Watch latest movies and shows free - lookmovie2


### OpenAI Call

In [10]:
def user_prompt_for(website):
    prompt = f'You are looking for a website titled: \"{website.title}\"\n\n'
    prompt += f'the content of this website is as follows:\n{website.content}\n\n'
    prompt += 'Provide a summary for this website in markdown'
    return user(prompt)

In [11]:
chat_history = [system('You are an assitant tool that analyzes a website content,\
                        and provides a summery. Ignore text that might be navigation related.\
                        Respond in Markdown.')]
pprint(chat_history)

[{'content': 'You are an assitant tool that analyzes a website '
             'content,                        and provides a summery. Ignore '
             'text that might be navigation related.                        '
             'Respond in Markdown.',
  'role': 'system'}]


In [12]:
chat_history.append(user_prompt_for(lookmovie))
pprint(chat_history)

[{'content': 'You are an assitant tool that analyzes a website '
             'content,                        and provides a summery. Ignore '
             'text that might be navigation related.                        '
             'Respond in Markdown.',
  'role': 'system'},
 {'content': 'You are looking for a website titled: "Watch latest movies and '
             'shows free - lookmovie2"\n'
             '\n'
             'the content of this website is as follows:\n'
             'Movies\n'
             'TV Shows\n'
             'filter\n'
             'Latest\n'
             'Categories\n'
             'Action\n'
             'Adventure\n'
             'Animation\n'
             'Comedy\n'
             'Crime\n'
             'Drama\n'
             'Documentary\n'
             'Sci-Fi\n'
             'Family\n'
             'History\n'
             'Fantasy\n'
             'Horror\n'
             'Music\n'
             'Mystery\n'
             'Romance\n'
             'Thriller\

In [13]:
completion = await cached_chat_completion(model = 'gpt-4o-mini', messages = chat_history)

In [14]:
Markdown(completion.choices[0].message.content)

# Website Summary: Watch Latest Movies and Shows Free - Lookmovie2

**Overview:**
Lookmovie2 is a free streaming platform that provides users access to a vast selection of movies and TV shows without the need for registration or payment. The site offers an extensive library with over 50,000 titles.

## Content:
- **Categories:** 
  - Action
  - Adventure
  - Animation
  - Comedy
  - Crime
  - Drama
  - Documentary
  - Sci-Fi
  - Family
  - History
  - Fantasy
  - Horror
  - Music
  - Mystery
  - Romance
  - Thriller
  - War
  - Western

- **Latest Added Movies:**
  - The Madden Cruiser Tour: A Bayou Adventure with Bill Belichick (2025) - Rating: 4.9/10
  - Dragon Wasps (2012) - Rating: 3.1/10
  - They Call Her Death (2024) - Rating: 5.6/10
  - Vini Jr. (2025) - Rating: 3.8/10
  - Modeled with Love (2023) - Rating: 6.0/10
  - Love in the Clouds (2025) - Rating: 5.2/10
  - Britain and the Blitz (2025) - Rating: 6.9/10
  - And many more...

## Features:
- Users can watch content without needing to sign up.
- Options to bookmark or share movies and episodes for later viewing.

## Additional Information:
The site provides a contact form for inquiries and mentions options for affiliate partnerships. 

**Disclaimer:** 
Please note that streaming copyrighted content may violate copyright laws in some jurisdictions.

In [15]:
chat_history.append(assistant(completion.choices[0].message.content))
pprint(chat_history)

[{'content': 'You are an assitant tool that analyzes a website '
             'content,                        and provides a summery. Ignore '
             'text that might be navigation related.                        '
             'Respond in Markdown.',
  'role': 'system'},
 {'content': 'You are looking for a website titled: "Watch latest movies and '
             'shows free - lookmovie2"\n'
             '\n'
             'the content of this website is as follows:\n'
             'Movies\n'
             'TV Shows\n'
             'filter\n'
             'Latest\n'
             'Categories\n'
             'Action\n'
             'Adventure\n'
             'Animation\n'
             'Comedy\n'
             'Crime\n'
             'Drama\n'
             'Documentary\n'
             'Sci-Fi\n'
             'Family\n'
             'History\n'
             'Fantasy\n'
             'Horror\n'
             'Music\n'
             'Mystery\n'
             'Romance\n'
             'Thriller\

Unexpected error occurred. Please check your request and contact support: https://langfuse.com/support.


In [16]:
chat_history.append(user('summerize all the movies and their ratings from the heighest to the lowest'))
pprint(chat_history)

[{'content': 'You are an assitant tool that analyzes a website '
             'content,                        and provides a summery. Ignore '
             'text that might be navigation related.                        '
             'Respond in Markdown.',
  'role': 'system'},
 {'content': 'You are looking for a website titled: "Watch latest movies and '
             'shows free - lookmovie2"\n'
             '\n'
             'the content of this website is as follows:\n'
             'Movies\n'
             'TV Shows\n'
             'filter\n'
             'Latest\n'
             'Categories\n'
             'Action\n'
             'Adventure\n'
             'Animation\n'
             'Comedy\n'
             'Crime\n'
             'Drama\n'
             'Documentary\n'
             'Sci-Fi\n'
             'Family\n'
             'History\n'
             'Fantasy\n'
             'Horror\n'
             'Music\n'
             'Mystery\n'
             'Romance\n'
             'Thriller\

In [17]:
completion = await cached_chat_completion(model = 'gpt-4o-mini', messages = chat_history)
completion.choices[0].message.content

Unexpected error occurred. Please check your request and contact support: https://langfuse.com/support.


'# Movie Ratings Summary\n\nHere is a list of the latest movies along with their ratings, organized from highest to lowest:\n\n1. **Maharaja (2024)** - Rating: **8.4/10**\n2. **ABBA: Against the Odds (2024)** - Rating: **7.3/10**\n3. **How Deep Is the Ocean (2023)** - Rating: **7.1/10**\n4. **Roy Orbison Forever (2022)** - Rating: **7.0/10**\n5. **Snow White (2025)** - Rating: **7.0/10**\n6. **Capturing Water (2024)** - Rating: **6.8/10**\n7. **Maybe Forever (2024)** - Rating: **6.6/10**\n8. **Steve Rannazzisi: Manchild (2013)** - Rating: **6.6/10**\n9. **I Want to Live on Mars (2023)** - Rating: **6.2/10**\n10. **The Featherweight (2023)** - Rating: **6.2/10**\n11. **Garbo: Where Did You Go? (2024)** - Rating: **6.0/10**\n12. **Modeled with Love (2023)** - Rating: **6.0/10**\n13. **The Sound of Waves (2023)** - Rating: **6.1/10**\n14. **Love in the Clouds (2025)** - Rating: **5.2/10**\n15. **Your Eyes on Me (2020)** - Rating: **5.5/10**\n16. **Kevin James Doyle: Fool Proof (2025)** - 

In [18]:
Markdown(completion.choices[0].message.content)

# Movie Ratings Summary

Here is a list of the latest movies along with their ratings, organized from highest to lowest:

1. **Maharaja (2024)** - Rating: **8.4/10**
2. **ABBA: Against the Odds (2024)** - Rating: **7.3/10**
3. **How Deep Is the Ocean (2023)** - Rating: **7.1/10**
4. **Roy Orbison Forever (2022)** - Rating: **7.0/10**
5. **Snow White (2025)** - Rating: **7.0/10**
6. **Capturing Water (2024)** - Rating: **6.8/10**
7. **Maybe Forever (2024)** - Rating: **6.6/10**
8. **Steve Rannazzisi: Manchild (2013)** - Rating: **6.6/10**
9. **I Want to Live on Mars (2023)** - Rating: **6.2/10**
10. **The Featherweight (2023)** - Rating: **6.2/10**
11. **Garbo: Where Did You Go? (2024)** - Rating: **6.0/10**
12. **Modeled with Love (2023)** - Rating: **6.0/10**
13. **The Sound of Waves (2023)** - Rating: **6.1/10**
14. **Love in the Clouds (2025)** - Rating: **5.2/10**
15. **Your Eyes on Me (2020)** - Rating: **5.5/10**
16. **Kevin James Doyle: Fool Proof (2025)** - Rating: **4.5/10**
17. **Trade (2019)** - Rating: **4.5/10**
18. **The Wedding Banquet (2025)** - Rating: **6.5/10**
19. **Vini Jr. (2025)** - Rating: **3.8/10**
20. **The Devil Comes to Kansas City (2023)** - Rating: **3.7/10**
21. **Catalyst (2025)** - Rating: **3.1/10**
22. **Dragon Wasps (2012)** - Rating: **3.1/10**
23. **The Arkansas Pigman Massacre (2025)** - Rating: **3.2/10**
24. **Cold War Steve Meets the Outside World (2020)** - Rating: **2.0/10**
25. **The Half-Breed (1916)** - Rating: **6.6/10**
26. **Another Gay Sequel: Gays Gone Wild! (2008)** - Rating: **4.0/10**
27. **Baywatch: Hawaiian Wedding (2003)** - Rating: **4.1/10**
28. **TinMan (2025)** - Rating: **3.4/10**
29. **The Madden Cruiser Tour: A Bayou Adventure with Bill Belichick (2025)** - Rating: **4.9/10**
30. **They Call Her Death (2024)** - Rating: **5.6/10**
31. **A Minecraft Movie (2025)** - Rating: **5.8/10**
32. **Luke Combs: Dive Bars to Country Star (2025)** - Rating: **2.0/10**
33. **Prof Tom Foolery Saves the Planet! (2017)** - Rating: **5.7/10**

This list includes all the movies mentioned and their respective ratings.

Unexpected error occurred. Please check your request and contact support: https://langfuse.com/support.
